# Introducción al Manejo de Memoria en LangChain

En LangChain, el manejo de memoria es un concepto clave que permite a las aplicaciones basadas en modelos de lenguaje realizar un seguimiento del historial de interacción. Esto resulta especialmente útil en escenarios donde el contexto de las interacciones anteriores debe ser conservado para proporcionar respuestas coherentes y relevantes.

### ¿Qué es la Memoria en LangChain?

El término "memoria" en LangChain se refiere a la capacidad de registrar y gestionar el historial de mensajes en una interacción, ya sea entre un usuario y un agente o en cualquier flujo conversacional. Este concepto es fundamental para aplicaciones como chatbots, asistentes virtuales y sistemas de soporte técnico, donde el contexto previo tiene un impacto significativo en la calidad de las respuestas generadas.

### Tipos de Memoria en LangChain

LangChain ofrece diferentes tipos de memoria, cada uno diseñado para adaptarse a necesidades específicas de almacenamiento y gestión del historial conversacional:

- **ChatMessageHistory**:
  - Permite guardar el historial de mensajes de un chat.
  - Proporciona métodos como `add_user_message` y `add_ai_message` para registrar manualmente los mensajes de usuario y del modelo.
--- 
- **ConversationBufferMemory**:
  - Al utilizar cadenas de tipo `ConversationalChain`, guarda automáticamente todos los mensajes de la conversación en un objeto de memoria.
  - Es ideal para escenarios donde es necesario conservar todo el historial de interacciones.
---
- **ConversationBufferWindowMemory**:
  - Similar a `ConversationBufferMemory`, pero permite definir una ventana de tamaño `k` para almacenar solo las últimas `k` interacciones en lugar de todo el historial.
  - Útil cuando se desea limitar la cantidad de contexto conservado para mejorar el rendimiento o reducir el consumo de recursos.
---
- **ConversationSummaryMemory**:
  - En lugar de almacenar los mensajes completos, genera un resumen del historial de la conversación.
  - Este enfoque reduce drásticamente el tamaño de la memoria, siendo ideal para conversaciones muy largas o aplicaciones con restricciones de almacenamiento.

### Importancia del Manejo de Memoria

El manejo de memoria en LangChain no solo permite mantener la coherencia en las respuestas, sino que también ayuda a optimizar los recursos y mejorar la eficiencia de las aplicaciones. La selección del tipo de memoria adecuado dependerá del caso de uso, el volumen de datos a manejar y los requiNotebook la aplicación.

Este tutorial profundizará en cómo implementar y utilizar estos diferentes tipos de memoria en LangChain, brindándote las herramientas necesarias para construir aplicaciones más inteligentes y contextualmente conscientes.


# Uso de ChatMessageHistory

- Importamos las librerías relacionadas y conectamos con el LLM

In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

# Cargar la API key desde .env
load_dotenv(override=True)

# Crear conexión con Groq
chat = ChatGroq(
    model="llama-3.3-70b-versatile",   # También puedes usar "mixtral-8x7b-32768"
    temperature=0.2
)

# Probar conexión
respuesta = chat.invoke("Hola, ¿cómo estás?, ¿quién eres?")
print(respuesta.content)

- Se instancia el objeto de histórico de mensajes

In [ ]:
from langchain.schema import SystemMessage, HumanMessage
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

- Realizamos la consulta del usuario

In [ ]:
consulta = "Hola, necesito asesoría para conectarme a la red wifi desde el PC"

- Se deben ir almacenando en el objeto "history" los mensajes de usuario y los mensajes AI que queramos

In [ ]:
history.add_user_message(consulta)

- Realizamos la consulta al LLM para posteriormente también guardar la respuesta

In [ ]:
resultado = chat.invoke([HumanMessage(content=consulta)])

In [ ]:
print(resultado.content)

In [ ]:
history.add_ai_message(resultado.content)

In [ ]:
history

In [ ]:
history.messages

In [ ]:
for mensaje in history.messages:
    print(f"{mensaje.type}: {mensaje.content}")

# Cómo crear un buffer de memoria de una conversación

## Clases `ConversationChain` y `ConversationBufferMemory` en LangChain

### `ConversationChain`
La clase `ConversationChain` es una cadena predefinida en LangChain diseñada para gestionar conversaciones interactivas con modelos de lenguaje. Su propósito es facilitar la construcción de flujos de diálogo, incorporando el manejo de entradas del usuario, generación de respuestas por parte del modelo y la integración de memoria para mantener el contexto a lo largo de la conversación.

#### Características principales:
- Funciona como un flujo básico para aplicaciones conversacionales.
- Es compatible con varios tipos de memoria para gestionar el historial de interacción.
- Ideal para prototipos rápidos o casos simples donde se requiere una conversación con contexto.

### `ConversationBufferMemory`
La clase `ConversationBufferMemory` es un tipo de memoria que guarda el historial completo de la conversación en forma de mensajes. Se integra perfectamente con `ConversationChain` para asegurar que el modelo tenga acceso al contexto completo de las interacciones previas.

#### Características principales:
- Almacena todos los mensajes de la conversación en orden cronológico.
- Útil para aplicaciones donde es crítico mantener el historial completo de interacciones.
- Proporciona métodos simples para recuperar o actualizar el historial de mensajes.

### Relación entre `ConversationChain` y `ConversationBufferMemory`
`ConversationChain` puede utilizar `ConversationBufferMemory` como su componente de memoria, lo que permite gestionar conversaciones con contexto completo de manera sencilla y eficiente.

Estas clases trabajan juntas para crear sistemas conversacionales que mantienen el historial y el contexto, mejorando la relevancia y coherencia de las respuestas.


In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


- Instanciamos el un objeto de la clase **ConversationBufferMemory**

In [ ]:
memory = ConversationBufferMemory()

- Se requiere para el ejemplo instanciar un objeto de la clase **ConversationChain**
- En ella se instancia la cadena conversacional con el LLM y el objeto de memoria
- Se mantiene verbose = True, para hacer seguimiento del proceso
(Una alternativa reciente: con *RunnableWithMessageHistory:* https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)

In [ ]:

conversation = ConversationChain(llm=chat,memory = memory,verbose=True)

- Se envía un primer prompt (human message)

In [ ]:

conversation.invoke(input="Hola, Sabes que equipos llegaron a la final en mundial de fútbol de Mexico en 1986. Respondeme en español")

- Se envía un segundo prompt (human message)

In [ ]:

conversation.predict(input="¿quienes eran los directores técnicos de cada selección?")

- Se puede observar el **histórico** de la conversación

In [ ]:
print(memory.buffer)

- Podemos cargar las variables historicas a la memoria

In [ ]:

memory.load_memory_variables({})

## Cómo salvar el buffer y posteriormente cargar 

- Recordemos en donde está la memoria de la conversación

In [ ]:
conversation.memory

### ¿Qué es la librería `pickle`?

La librería `pickle` es un módulo estándar de Python que permite serializar y deserializar objetos. **Serializar** significa convertir un objeto de Python en una secuencia de bytes que puede ser almacenada en un archivo o transmitida a través de una red. **Deserializar** significa reconstruir el objeto original a partir de esa secuencia de bytes.

### Usos principales
- **Almacenamiento de datos**: Guardar estructuras de datos complejas, como listas, diccionarios o clases personalizadas, para su uso posterior.
- **Transferencia de datos**: Enviar objetos entre diferentes sistemas o procesos.
- **Persistencia de estados**: Guardar el estado de un programa o modelo para reanudarlo en otro momento.

### Limitaciones
- Solo es compatible con objetos de Python.
- Puede ser inseguro si se carga un archivo pickle de una fuente no confiable, ya que podría ejecutar código malicioso.




In [ ]:
import pickle

# Vamos a crear un objeto binario con todo el objeto de la memoria
pickled_str = pickle.dumps(conversation.memory) 

- Se cfrea un archivo binario para guardar la conversacion

In [ ]:
 # Utilizamos wb para indicar que escriba un objeto binario, en la misma ruta que el script

with open('memory.pkl','wb') as f:
    f.write(pickled_str)

### Cargar la memoria

In [ ]:
# Utilizamos rb para indicar que leemos el objeto binario
memoria_cargada = open('memory.pkl','rb').read() 

## Creamos una conversación nueva

- Se creamos una nueva instancia de LLM para asegurar que está totalmente limpia

In [ ]:

# Crear otra conexión con Groq
chat2 = ChatGroq(
    model="llama-3.3-70b-versatile",   # También puedes usar "mixtral-8x7b-32768"
    temperature=0.2
)

In [ ]:
conversacion_recargada = ConversationChain(
    llm=chat2, 
    memory = pickle.loads(memoria_cargada),
    verbose=True
)

- Vamos a verificar la memoria de la nueva conversación

In [ ]:
conversacion_recargada.memory.buffer

In [ ]:
conversation.predict(input="¿qué equipos participaron en las semifinales?")

## Crear un buffer con ventana de memoria

- Como siempre, carguemos las liberías involucradas y el modelo

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory


- Se va a instanciar un objeto **ConversationBufferWindowMemory**, donde *k* indica el número de iteraciones (pareja de mensajes human-AI) que guardar

In [ ]:
memory = ConversationBufferWindowMemory(k=2) 

- Ahora se instancia  una cadena conversacional con el LLM y el objeto de memoria.

    alternativa usar RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

In [ ]:
#Creamos una instancia de la cadena conversacional con el LLM y el objeto de memoria
conversation = ConversationChain(llm=chat,memory = memory,verbose=True)


In [ ]:
conversation.predict(input="Hola, soy Juan, ¿puedes ayudarme en algo?")

- Continuamos con la conversación

In [ ]:
conversation.predict(input="Quiero hacer una consulta sobre la historia reciente de Colombia")

In [ ]:
conversation.predict(input="¿Sabes que presidentes tuvo el pais entre 1980 y 1990?")

In [ ]:
print(memory.buffer) #k limita el número de interacciones

In [ ]:
conversation.predict(input="¿A qué partidos políticos pertenecían dichos presidentes?")

In [ ]:
print(memory.buffer) #k limita el número de interacciones

## Manejar una conversación 'resumida' en la memoria

- Para lo cual vamos a usar la clase *ConversationSummaryBufferMemory*

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


- Instanciamos el objeto

(Una versión actualizada: https://python.langchain.com/docs/versions/migrating_memory/)

In [ ]:
# Si genera error, es posible requiera actualizar 
# pip install --upgrade langchain pydantic

In [ ]:
memory = ConversationSummaryBufferMemory(llm=chat)

- **Creemos una plantilla para dar contexto a la conversaión**

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "Responde siempre en español."),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])


- Se crea un prompt cuya respuesta hará que se sobrepase el límite de tokens y por tanto sea recomendable resumir la memoria

In [ ]:
proceso_votacion = '''Según el reglamento académico, para sustentar el trabajo de grado en modalidad practica empresarial, se requiere certificado de la empresa de terminación de la práctica'''

- Ahora se crea una conversación con memoria resumida

Alternativa con RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

In [ ]:
memory = ConversationSummaryBufferMemory(
    llm=chat,
    memory_key="history",
    return_messages=True,          # <<--- IMPORTANTE
    max_token_limit=100
)



In [ ]:
conversation = ConversationChain(llm=chat,memory = memory, prompt = prompt, verbose=True)

In [ ]:
conversation.invoke({"input":proceso_votacion})

- Revisemos que se está almacenando en la memoria de la conversación

In [ ]:
#memory.load_memory_variables({}) 

In [ ]:
print(memory.buffer)

In [ ]:
pregunta2 = "¿para qué necesito ese certificado?"
respuesta = conversation.invoke(input=pregunta2)

In [ ]:
print(memory.buffer)

In [ ]:
respuesta

In [ ]:
respuesta['response']